In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset



In [17]:
# Load your encoded dataset from Excel or generate example data
# Replace "your_encoded_dataset.xlsx" with the path to your encoded Excel file
df = pd.read_excel("/content/resampled_dataset.xlsx")
df.head()
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows
# how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
df['Result']= label_encoder.fit_transform(df['Result'])

df['Result'].unique()
df.head()

,Address_Encode,House,Street,API,Result
0,0,1,0,1,1
1,1,0,1,0,0
2,1,1,0,0,0
3,0,1,0,0,0
4,0,1,1,0,0


In [18]:
# Extract encoded features and labels from the DataFrame
# Replace column names with the appropriate ones from your dataset
encoded_features = df[['Address_Encode','House','Street','API']].values  # Replace with your column names
labels = df['Result'].values



In [19]:
# Convert encoded features and labels to PyTorch tensors
encoded_features = torch.tensor(encoded_features, dtype=torch.float32)  # Adjust dtype if needed
labels = torch.tensor(labels, dtype=torch.long)



In [20]:
# Split the data into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(encoded_features, labels, test_size=0.2, random_state=42)

# Create DataLoader for training and testing sets
train_dataset = TensorDataset(features_train, labels_train)
test_dataset = TensorDataset(features_test, labels_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)



In [21]:
# Define the RNN model
class SimpleRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out



In [22]:
# Instantiate the model
num_classes = 3
input_dim = encoded_features.shape[1]  # Adjust this based on the number of encoded columns
hidden_dim = 64
output_dim = num_classes  # Define num_classes
model = SimpleRNN(input_dim, hidden_dim, output_dim)



In [23]:
# Set up the optimizer and loss function
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()



In [24]:
# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        inputs, labels = batch
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()



IndexError: ignored

In [ ]:
# Evaluation
model.eval()
total_correct = 0
total_samples = 0
with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs = model(inputs)
        predicted_labels = torch.argmax(outputs, dim=1)
        total_correct += torch.sum(predicted_labels == labels).item()
        total_samples += len(labels)
accuracy = total_correct / total_samples



In [ ]:
print("Test Accuracy:", accuracy)